# ROS 2 - Apuntes: Parámetros y Launch Files

## 1. Objetivo
En este bloque aprenderemos a hacer que nuestros nodos sean flexibles y configurables sin necesidad de recompilar el código.

- Entender qué es un **Parámetro** en ROS 2.
- Aprender a declarar y leer parámetros desde Python (`rclpy`).
- Introducir los **Launch Files**, la herramienta para lanzar múltiples nodos de forma organizada.

## 2. Analogía: El mando de la televisión
Imagina un nodo que controla el brillo de una pantalla. Si escribes `brillo = 50` en el código, para cambiarlo tendrías que modificar el archivo y compilar. 
Un **parámetro** es como el menú de ajustes: el programa está corriendo y tú, desde fuera (CLI o Launch file), le dices: "ahora el brillo es 80".

## 3. Manejo de Parámetros en Python
Un nodo de ROS 2 puede tener sus propios parámetros (variables locales accesibles externamente).

### Código Fuente: Nodo con Parámetros
A continuación, un ejemplo de un nodo que imprime un mensaje un número determinado de veces, ambos valores configurables.

```python
import rclpy
from rclpy.node import Node

class ParamNode(Node):
    def __init__(self):
        super().__init__('nodo_configurable')
        
        # 1. Declarar parámetros (Nombre, Valor por defecto)
        self.declare_parameter('mensaje', 'Hola Mundo')
        self.declare_parameter('frecuencia', 1.0)
        
        # 2. Leer parámetros
        msg = self.get_parameter('mensaje').get_parameter_value().string_value
        timer_period = 1.0 / self.get_parameter('frecuencia').get_parameter_value().double_value
        
        self.get_logger().info(f'Iniciando nodo con mensaje: {msg}')
        self.timer = self.create_timer(timer_period, self.timer_callback)

    def timer_callback(self):
        # Re-leer por si ha cambiado dinámicamente
        actual_msg = self.get_parameter('mensaje').get_parameter_value().string_value
        self.get_logger().info(f'Publicando: {actual_msg}')

def main(args=None):
    rclpy.init(args=args)
    node = ParamNode()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()
```

### Comandos CLI para Parámetros
- Listar parámetros de un nodo: `ros2 param list /nodo_configurable`
- Obtener un valor: `ros2 param get /nodo_configurable mensaje`
- Cambiar un valor en caliente: `ros2 param set /nodo_configurable mensaje "Adiós Mundo"`

## 4. Launch Files: El "Play" del sistema
En robótica no lanzamos 20 terminales a mano. Usamos archivos `.launch.py` para arrancar todo el sistema con una sola instrucción.

### Ejemplo de Launch File simple
Archivo: `mi_lanzamiento_launch.py`
```python
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='turtlesim',
            executable='turtlesim_node',
            name='simulador_ventanal'
        ),
        Node(
            package='mi_paquete',
            executable='mi_nodo_param',
            parameters=[{'mensaje': 'Tortuga en marcha', 'frecuencia': 2.0}]
        )
    ])
```

## 5. Ejercicios de Consolidación
1. **Reto de Velocidad**: Crea un nodo que se suscriba a `/turtle1/pose` y que tenga un parámetro llamado `umbral_distancia`. Si la tortuga supera esa distancia desde el origen, el nodo debe mostrar un aviso.
2. **Launch Maestro**: Crea un launch file que arranque `turtlesim_node` y dos instancias de tu nodo de parámetros con nombres diferentes (`nodo_a` y `nodo_b`) y mensajes distintos.

> **Tip de Experto**: Si cambias un parámetro con `ros2 param set`, el cambio es volátil. Para que sea permanente, debes incluirlo en el Launch file o cargarlo desde un archivo `.yaml`.

***Fichero `setup.py`***

```bash
from setuptools import find_packages, setup
import os           # <--- Añadir
from glob import glob # <--- Añadir

package_name = 'mi_primer_paquete'

setup(
    name=package_name,
    version='0.0.0',
    packages=find_packages(exclude=['test']),
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
        # --- NUEVA LÍNEA PARA LOS LAUNCH FILES ---
        # Esto indica que todos los archivos en la carpeta 'launch/'
        # deben copiarse a 'share/mi_primer_paquete/launch' al compilar
        (os.path.join('share', package_name, 'launch'), glob(os.path.join('launch', '*launch.[pxy][yma]*'))),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='Jordi Pozo',
    maintainer_email='jpozoc@gmail.com',
    description='TODO: Package description',
    license='TODO: License declaration',
    extras_require={
        'test': [
            'pytest',
        ],
    },
    entry_points={
        'console_scripts': [
            # --- REGISTRO DEL NODO ---
            # 'nombre_ejecutable = paquete.archivo_python:funcion_main'
            'mi_nodo_param = mi_primer_paquete.mi_nodo_param:main',
        ],
    },
)
    

```

***Fichero `package.xml`***

```xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>mi_primer_paquete</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="jpozoc@gmail.com">Jordi Pozo</maintainer>
  <license>TODO: License declaration</license>

  <exec_depend>rclpy</exec_depend>
  <exec_depend>launch_ros</exec_depend>

  <test_depend>ament_copyright</test_depend>
  <test_depend>ament_flake8</test_depend>
  <test_depend>ament_pep257</test_depend>
  <test_depend>ament_xmllint</test_depend>
  <test_depend>python3-pytest</test_depend>

  <export>
    <build_type>ament_python</build_type>
  </export>
</package>
```